# Taller 3: Exploratory Data Analysis (EDA)

**Dataset:** Forbes Global 2000 (2025)  
**Archivo:** `Forbes_2000_Companies_2025.csv`  
**Generado:** 2025-09-03 00:02:11

Este cuaderno realiza un EDA completo siguiendo los pasos solicitados.


## 1. Entender la necesidad

**Objetivo:** Comprender los factores asociados al tamaño y desempeño financiero de las compañías del ranking Forbes Global 2000 (2025).  
Preguntas guía:
- ¿Cómo se distribuyen las métricas financieras clave?  
- ¿Qué países e industrias concentran más compañías y mayor valor?
- ¿Qué relaciones existen entre variables financieras?
- ¿Qué métricas derivadas ayudan a interpretar el desempeño?


## 2. Descargar y abrir los datos

El archivo CSV ya está disponible localmente. Se carga con **pandas**.


In [ ]:
# === 2) Carga de datos ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}" if isinstance(x, float) else str(x))

file_path = r"/mnt/data/Forbes_2000_Companies_2025.csv"

def load_csv_robust(path):
    try:
        return pd.read_csv(path, sep=None, engine="python", encoding="utf-8")
    except Exception:
        pass
    try:
        return pd.read_csv(path, sep=None, engine="python", encoding="utf-8-sig")
    except Exception:
        pass
    for enc in ["utf-8", "utf-8-sig", "latin-1"]:
        for sep in [",", ";", "\t", "|"]:
            try:
                return pd.read_csv(path, sep=sep, encoding=enc)
            except Exception:
                continue
    try:
        return pd.read_csv(path, sep=",", encoding="utf-8", on_bad_lines="skip")
    except Exception:
        return None

raw = load_csv_robust(file_path)
assert raw is not None, "No se pudo leer el CSV. Revisa el separador/encoding."
print("Shape:", raw.shape)
raw.head()


## 3. Limpieza de la base de datos

- Estandarizar nombres de columnas a **snake_case**.  
- Remover duplicados.  
- Convertir a numérico columnas con símbolos (`$`, `,`, etc.).  
- Resumen de valores faltantes.


In [ ]:
# === 3) Limpieza ===
def to_snake(s):
    return (
        str(s).strip()
        .replace("/", " ")
        .replace("-", " ")
        .replace(".", " ")
        .replace("(", " ").replace(")", " ")
        .replace("%", "pct")
        .lower()
        .replace(" ", "_")
    )

df = raw.copy()
df.columns = [to_snake(c) for c in df.columns]

before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"Duplicados eliminados: {before - after}")

def is_probably_numeric(series: pd.Series) -> bool:
    if series.dtype != object:
        return False
    sample = series.dropna().astype(str).head(50).str.strip()
    if sample.empty:
        return False
    hits = sample.str.contains(r"[0-9]")
    return hits.mean() > 0.6

def coerce_numeric(col: pd.Series) -> pd.Series:
    cleaned = (
        col.astype(str)
        .str.replace(r"[\\$,]", "", regex=True)
        .str.replace(r"\\s+", "", regex=True)
        .str.replace(r"(B|M)$", "", regex=True)
    )
    return pd.to_numeric(cleaned, errors="coerce")

for c in df.columns:
    if is_probably_numeric(df[c]):
        converted = coerce_numeric(df[c])
        if converted.notna().sum() >= max(3, int(0.3 * len(converted))):
            df[c] = converted

nulls = df.isna().sum().sort_values(ascending=False)
nulls


## 4. Transformaciones (10%)

Cálculo de métricas derivadas si existen columnas necesarias:
- `profit_margin = profits / sales`
- `market_to_assets = market_value / assets`
- `asset_turnover = sales / assets`


In [ ]:
# === 4) Transformaciones ===
def find_col(alias_list):
    for col in df.columns:
        for a in alias_list:
            if a in col:
                return col
    return None

sales_col   = find_col(["sales", "revenue"])
profits_col = find_col(["profit", "net_income"])
assets_col  = find_col(["asset"])
mv_col      = find_col(["market_value", "market_cap"])

if profits_col and sales_col:
    df["profit_margin"] = df[profits_col] / df[sales_col]
if mv_col and assets_col:
    df["market_to_assets"] = df[mv_col] / df[assets_col]
if sales_col and assets_col:
    df["asset_turnover"] = df[sales_col] / df[assets_col]

df.head()


## 5. Exploratorio Inicial (40%)

Incluye:
- Estadísticas descriptivas de variables numéricas.  
- Histogramas (univariado).  
- Conteos por categorías principales (país/industria si existen).  
- Dispersión (bivariado) y matriz de correlación.  


In [ ]:
# === 5.1 Descriptivas ===
numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
df[numeric_cols].describe().T if numeric_cols else pd.DataFrame()


In [ ]:
# === 5.2 Univariado: histogramas ===
import matplotlib.pyplot as plt

to_plot = numeric_cols[:6]  # limitar a 6 para evitar exceso
for c in to_plot:
    plt.figure()
    plt.hist(df[c].dropna(), bins=30)
    plt.title(f"Histograma: {c}")
    plt.xlabel(c); plt.ylabel("Frecuencia")
    plt.show()

# Categóricas (industry/sector/country) si existen
cat_cols = [c for c in df.columns if df[c].dtype == object]
def first_match(aliases):
    for col in cat_cols:
        for a in aliases:
            if a in col:
                return col
    return None

industry_col = first_match(["industry", "sector"])
country_col  = first_match(["country"])

for target in [industry_col, country_col]:
    if target:
        counts = df[target].value_counts().head(15)
        plt.figure()
        counts.plot(kind="bar")
        plt.title(f"Top 15 {target} por frecuencia")
        plt.xlabel(target); plt.ylabel("Conteo")
        plt.tight_layout()
        plt.show()


In [ ]:
# === 5.3 Bivariado: scatter ===
def resolve_col(alias_options):
    for col in df.columns:
        for alias in alias_options:
            if alias in col:
                return col
    return None

assets_real = resolve_col(["asset"])
mv_real     = resolve_col(["market_value", "market_cap"])
sales_real  = resolve_col(["sales", "revenue"])
profits_real= resolve_col(["profit", "net_income"])

def scatter_if_exists(xc, yc, title):
    if xc and yc and pd.api.types.is_numeric_dtype(df[xc]) and pd.api.types.is_numeric_dtype(df[yc]):
        plt.figure()
        plt.scatter(df[xc], df[yc], alpha=0.6)
        plt.title(title)
        plt.xlabel(xc); plt.ylabel(yc)
        plt.tight_layout()
        plt.show()

scatter_if_exists(assets_real, mv_real, f"Scatter: {assets_real} vs {mv_real}")
scatter_if_exists(sales_real, profits_real, f"Scatter: {sales_real} vs {profits_real}")


In [ ]:
# === 5.4 Correlaciones ===
if len(numeric_cols) >= 2:
    corr = df[numeric_cols].corr(numeric_only=True)
    plt.figure(figsize=(8,6))
    plt.imshow(corr, aspect='auto')
    plt.colorbar()
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.index)), corr.index)
    plt.title("Matriz de correlaciones (numéricas)")
    plt.tight_layout()
    plt.show()

    corr_abs = corr.abs().copy()
    corr_abs.values[np.tril_indices_from(corr_abs)] = np.nan
    top_pairs = (
        corr_abs.unstack()
        .dropna()
        .sort_values(ascending=False)
        .head(10)
    )
    print("Top correlaciones absolutas:")
    display(top_pairs)
else:
    print("No hay suficientes columnas numéricas para una correlación útil.")


In [ ]:
# === 5.5 Resúmenes por grupo (industry/country) ===
def first_match_col(aliases):
    for col in df.columns:
        for a in aliases:
            if a in col:
                return col
    return None

industry_col = first_match_col(["industry", "sector"])
country_col  = first_match_col(["country"])

metrics = [c for c in ["sales","profits","assets","market_value","profit_margin","market_to_assets","asset_turnover"] if c in df.columns]
if industry_col and metrics:
    display(df.groupby(industry_col)[metrics].mean(numeric_only=True).sort_values(by=metrics[0], ascending=False).head(15))
if country_col and metrics:
    display(df.groupby(country_col)[metrics].mean(numeric_only=True).sort_values(by=metrics[0], ascending=False).head(15))


## 6. Análisis y conclusiones (50%)

Interpreta los resultados, responde las preguntas y propone recomendaciones.  
Abajo se genera un **borrador** de hallazgos automáticos para que lo ajustes con tu criterio.


In [ ]:
# === Borrador de conclusiones automáticas ===
findings = []

# Correlaciones destacadas
if 'corr' in locals():
    corr_copy = corr.copy()
    corr_copy.values[np.tril_indices_from(corr_copy)] = np.nan
    top = corr_copy.unstack().dropna().sort_values(ascending=False).head(5)
    if not top.empty:
        findings.append("**Top correlaciones (lineales) entre variables numéricas:**")
        for (a,b), v in top.items():
            findings.append(f"- {a} ↔ {b}: {v:.3f}")

# Industrias/países si existen
def fm(aliases):
    for col in df.columns:
        for a in aliases:
            if a in col:
                return col
    return None

industry_col = fm(["industry","sector"])
country_col  = fm(["country"])
mv_col       = fm(["market_value","market_cap"])
pm_col       = "profit_margin" if "profit_margin" in df.columns else None

def top_group(colname, metric, k=5):
    if colname and metric and colname in df.columns and metric in df.columns:
        t = df.groupby(colname)[metric].mean(numeric_only=True).sort_values(ascending=False).head(k)
        return t
    return None

if mv_col:
    top_ind = top_group(industry_col, mv_col, 5)
    top_cty = top_group(country_col, mv_col, 5)
    if top_ind is not None and len(top_ind) > 0:
        findings.append(f"**Industrias con mayor {mv_col} promedio (Top 5):**")
        for idx, val in top_ind.items():
            findings.append(f"- {idx}: {val:,.2f}")
    if top_cty is not None and len(top_cty) > 0:
        findings.append(f"**Países con mayor {mv_col} promedio (Top 5):**")
        for idx, val in top_cty.items():
            findings.append(f"- {idx}: {val:,.2f}")

if pm_col and industry_col:
    pm_ind = df.groupby(industry_col)[pm_col].mean(numeric_only=True).sort_values(ascending=False).head(5)
    if len(pm_ind) > 0:
        findings.append("**Industrias con mayor margen de utilidad promedio (Top 5):**")
        for idx, val in pm_ind.items():
            findings.append(f"- {idx}: {val:.2%}")

from IPython.display import Markdown
Markdown("\\n".join(findings) if findings else "Agrega aquí tus conclusiones basadas en el análisis.")


---

### Notas
- Ajusta la variable `file_path` si el CSV cambia de ruta.  
- Todos los gráficos usan **matplotlib** sin estilos ni colores específicos (y un gráfico por figura).  
- Puedes extender el EDA con segmentaciones, outliers, o modelos simplificados.
